In [1]:
import glob

import geopandas as gpd
import momepy as mm
import numpy as np

regions_datadir = "/data/uscuni-ulce/"
data_dir = "/data/uscuni-ulce/processed_data/"
eubucco_files = glob.glob(regions_datadir + "eubucco_raw/*")
graph_dir = data_dir + "neigh_graphs/"

In [2]:
region_hulls = gpd.read_parquet(regions_datadir + "regions/" + "regions_hull.parquet")

In [3]:
# 12199 - hills, small test
# 69300 - prague medium
# 226 - germany somewhere, largest cluster
# 106928 + 1 - big one in poland
# 106928 - small test
for region_id, region_hull in region_hulls.iterrows():
    if region_id < 226:
        continue
    break
region_id

226

In [6]:
%%time
df_streets = gpd.read_parquet(data_dir + f"streets/streets_{region_id}.parquet")
df_buildings = gpd.read_parquet(data_dir + f"buildings/buildings_{region_id}.parquet")
df_streets.shape, df_buildings.shape

CPU times: user 8 µs, sys: 5 µs, total: 13 µs
Wall time: 13.4 µs


((1313900, 2), (3642166, 3))

In [ ]:
%%time
start = np.datetime64("now")
res = mm.street_profile(df_streets, df_buildings)
end = np.datetime64("now")

In [13]:
end - start

numpy.timedelta64(409,'s')

In [15]:
res10 = res.copy()

In [16]:
%%time
start = np.datetime64("now")
res = mm.street_profile(df_streets, df_buildings, distance=3)
end = np.datetime64("now")

CPU times: user 15min 44s, sys: 7.31 s, total: 15min 51s
Wall time: 15min 51s


In [17]:
end - start

numpy.timedelta64(952,'s')

In [14]:
%%time
graph = mm.gdf_to_nx(df_streets)
graph = mm.node_degree(graph)

CPU times: user 29 s, sys: 915 ms, total: 29.9 s
Wall time: 29.8 s


In [15]:
%%time
graph = mm.subgraph(
    graph,
    radius=5,
    meshedness=True,
    cds_length=False,
    mode="sum",
    degree="degree",
    length="mm_len",
    mean_node_degree=False,
    proportion={0: True, 3: True, 4: True},
    cyclomatic=False,
    edge_node_ratio=False,
    gamma=False,
    local_closeness=True,
    closeness_weight="mm_len",
    verbose=True,
)

  0%|          | 0/1847875 [00:00<?, ?it/s]

CPU times: user 3min 57s, sys: 1.28 s, total: 3min 58s
Wall time: 3min 57s


In [16]:
%%time
graph = mm.cds_length(graph, radius=3, name="ldsCDL", verbose=False)
graph = mm.clustering(graph, name="xcnSCl")
graph = mm.mean_node_dist(graph, name="mtdMDi", verbose=False)

CPU times: user 2min 38s, sys: 900 ms, total: 2min 39s
Wall time: 2min 39s


In [17]:
nodes, edges = mm.nx_to_gdf(graph, spatial_weights=False)

In [18]:
gpd.__version__

'1.0.0-alpha1+39.ge829498'